In [28]:
import numpy as np
import math
import csv
import time
#from google.colab import drive
np.set_printoptions(suppress=True) # 顯示小數點
#drive.mount('/content/drive')

In [29]:
#cd drive/My Drive/CSIE/Machine Learning

In [30]:
#pwd

In [31]:
raw_data = np.genfromtxt('train.csv', delimiter=',', encoding='big5') ## load train.csv
data = raw_data[1:, 3:]  # data[10]為PM2.5

where_are_NaNs = np.isnan(data)
data[where_are_NaNs] = 0.0   # 將nan轉為數值0.0
  
x = np.zeros((1, 24*20*12))  
for row in range(18):  # 把所有日子的氣象資料整理為形狀(18, -1)的矩陣    
    data_term = np.array(data[row, :].reshape(1, 24))
    while(row+18 < len(data)):        
        row += 18    
        data_term = np.append(data_term, data[row,  : ].reshape(1, 24), axis = 1) 
    x = np.append(x, data_term, axis = 0)
x = np.delete(x, 0, 0)

In [32]:
train_x = [ ]
train_y = [ ]

for month in range(12): 
    for day in range(20):    
        for hr in range(24):   
            if(day != 19) or (hr < 15): # 撇除無法用來預測的data
            # 將每九個小時的十八種類別資料，存為一筆一維陣列，每個月有480筆資料(每個月20天，每天24筆)
                train_x.append(x[ : , (month * 480 + day * 24 + hr) : (month * 480 + day * 24 + hr + 9)])

            if(day != 0) or (hr > 8):  # 撇除無法作為被預測的data（第十小時）
                train_y.append(x[9, (month * 480 + day * 24 + hr)])  # 可用前九小時的data來預測的PM2.5

In [33]:
print('shape of train_x :', np.array(train_x).shape)
print('shape of train_y(10-th hour PM2.5) :', np.array(train_y).shape)
print('model : y = b + X*w')
print('Loss function : Loss = summation(y\' - y)^2' )

shape of train_x : (5652, 18, 9)
shape of train_y(10-th hour PM2.5) : (5652,)
model : y = b + X*w
Loss function : Loss = summation(y' - y)^2


In [34]:
train_x = np.array(train_x)
train_x = train_x.reshape(train_x.shape[0], -1) # rehshape into (5652, 18 *9)
train_y = np.array(train_y)
train_y = train_y.reshape(train_y.shape[0], -1) # rehshape into (5652, 1)

# Nolmalization
mean = np.mean(train_x, axis = 0)
std = np.std(train_x, axis = 0)
for i in range(train_x.shape[0]):
    for j in range(train_x.shape[1]):
        if not std[j] == 0 :
            train_x[i][j] = (train_x[i][j]- mean[j]) / std[j]

b = 1.0
w = np.zeros((train_x.shape[1], 1))

b_history = np.array(b)
w_history = np.array(w.reshape(1, -1))

b_grad_history = b_history
w_grad_history = w_history

# using AdaGrad
lr = 1.0   # learning rate
lr_b = 1.0  # customized learning rate for b
lr_w = 1.0  # customized learning rate for w
iteration = 10000

t_start = time.time()
for i in range(iteration):      
    b_grad = np.sum((-2.0) * (train_y - b - train_x.dot(w)))  # train_x.shape[0] = 5652
    w_grad = (-2.0) * (np.transpose(train_x).dot(train_y - b - train_x.dot(w)))

    lr_b += b_grad ** 2.0
    lr_w += w_grad ** 2.0

    b -= lr * b_grad / (np.sqrt(lr_b) + 0.00000001)
    w -= lr * w_grad / (np.sqrt(lr_w) + 0.00000001)

    b_history = np.append(b_history, b)
    w_history = np.append(w_history, w.reshape(1, -1), axis = 0)

    b_grad_history = np.append(b_grad_history, b_grad)
    w_grad_history = np.append(w_grad_history, w_grad.reshape(1, -1), axis = 0)
  
    if(i % 500 == 499):
        t_end = time.time()
        print('-\nIteration[%d]:\nb = %.9f, b_grad = %.9f, mean of w = %.6f, mean of w_grad = %.6f' % (i + 1, b, b_grad, np.mean(w), np.mean(w_grad)))    
        print("Loss:", np.power(np.sum(np.power(train_x.dot(w) - b - train_y, 2 ))/ train_x.shape[0], 0.5))
        #print("Loss:", np.sum(np.power(train_y - b - train_x.dot(w), 2 )))
        print('Elapsed time: %.2f min.' % ((t_end - t_start) / 60.0)) 

-
Iteration[500]:
b = 21.189479457, b_grad = -2103.994466918, mean of w = 0.097032, mean of w_grad = -4.362472
Loss: 42.989591399814955
Elapsed time: 0.01 min.
-
Iteration[1000]:
b = 21.371432636, b_grad = -29.576668441, mean of w = 0.096980, mean of w_grad = -1.485474
Loss: 43.141342004626274
Elapsed time: 0.02 min.
-
Iteration[1500]:
b = 21.373990423, b_grad = -0.415786016, mean of w = 0.096742, mean of w_grad = -0.716006
Loss: 43.13394758489729
Elapsed time: 0.04 min.
-
Iteration[2000]:
b = 21.374026380, b_grad = -0.005845081, mean of w = 0.096514, mean of w_grad = -0.410120
Loss: 43.12937356691667
Elapsed time: 0.07 min.
-
Iteration[2500]:
b = 21.374026886, b_grad = -0.000082170, mean of w = 0.096321, mean of w_grad = -0.260107
Loss: 43.12701479043524
Elapsed time: 0.10 min.
-
Iteration[3000]:
b = 21.374026893, b_grad = -0.000001155, mean of w = 0.096153, mean of w_grad = -0.175666
Loss: 43.12575264576659
Elapsed time: 0.13 min.
-
Iteration[3500]:
b = 21.374026893, b_grad = -0.0000

In [35]:
# save the training result into .csv files
with open('b_history.csv', 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile)  # 建立 CSV 檔寫入器
    writer.writerow(b_history)    # 寫入一列資料

with open('w_history.csv', 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile)
    writer.writerows(w_history)

with open('b_grad_history.csv', 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile)
    writer.writerow(b_grad_history)

with open('w_grad_history.csv', 'w', newline='') as csvfile:  
    writer = csv.writer(csvfile)
    writer.writerows(w_grad_history)

In [36]:
# load the training result
b_result = np.genfromtxt('b_history.csv', delimiter=',', encoding='big5')
b_grad_result = np.genfromtxt('b_grad_history.csv', delimiter=',', encoding='big5')
w_result = np.genfromtxt('w_history.csv', delimiter=',', encoding='big5')
w_grad_result = np.genfromtxt('w_grad_history.csv', delimiter=',', encoding='big5')

In [38]:
print('Model : y = x * w + b')
# predict the PM2.5 of training data

#Average_ERROR = np.array([10000.0])  # a random initial value for average error comparision
Average_ERROR = 10000.0  # a random initial value for average error comparision
t_start = time.time()
for c in range(0, len(b_history), 50):    # to find the index, which its average error is the smallest.
    prediction_train = np.array([])
    for i in range(len(train_x)):
        predict_value = sum(train_x[i] * w_history[c]) + b_history[c]
        if(predict_value < 0):  # when prediction is a negative number, set it to 0.0
            predict_value = 0.0
        prediction_train = np.append(prediction_train, predict_value)
    ERROR = abs(sum(prediction_train - train_y).mean())  
    if (ERROR < Average_ERROR):  # if current error is smaller, then update the average error.
        Average_ERROR = ERROR
        result_index = c
    if (c % 500 == 0):
        t_end = time.time()
        print('-\nIteration %d: Current Best Average_ERROR = %.5f' % (c, Average_ERROR))
        print('Elapsed time: %.2f min.' % ((t_end - t_start) / 60.0))
print('-\nFinal index %d, with smallest AVERAGE ERROR : %.5f' % (result_index, Average_ERROR))
b_final = b_history[result_index]  # the best-performed b value
w_final = w_history[result_index]  # the best-performed w value

Model : y = x * w + b
-
Iteration 0: Current Best Average_ERROR = 10000.00000
Elapsed time: 0.00 min.
-
Iteration 500: Current Best Average_ERROR = 979.28655
Elapsed time: 0.05 min.
-
Iteration 1000: Current Best Average_ERROR = 13.06683
Elapsed time: 0.10 min.
-
Iteration 1500: Current Best Average_ERROR = 13.06683
Elapsed time: 0.15 min.
-
Iteration 2000: Current Best Average_ERROR = 13.06683
Elapsed time: 0.19 min.
-
Iteration 2500: Current Best Average_ERROR = 13.06683
Elapsed time: 0.24 min.
-
Iteration 3000: Current Best Average_ERROR = 13.06683
Elapsed time: 0.29 min.
-
Iteration 3500: Current Best Average_ERROR = 13.06683
Elapsed time: 0.34 min.
-
Iteration 4000: Current Best Average_ERROR = 13.06683
Elapsed time: 0.39 min.
-
Iteration 4500: Current Best Average_ERROR = 13.06683
Elapsed time: 0.43 min.
-
Iteration 5000: Current Best Average_ERROR = 13.06683
Elapsed time: 0.48 min.
-
Iteration 5500: Current Best Average_ERROR = 13.06683
Elapsed time: 0.53 min.
-
Iteration 6000: 

In [41]:
print('Model : y = x * w + b')
# predict the PM2.5 of training data
prediction_train = np.array([])
for i in range(len(train_x)):
    predict_value = sum(train_x[i] * w_final) + b_final
    if(predict_value < 0):
        predict_value = 0.0
    prediction_train = np.append(prediction_train, predict_value)

print('Average ERROR on Training Data:', sum(prediction_train - train_y))
training_data_prediction = prediction_train

Model : y = x * w + b
Average ERROR on Training Data: [  1165.67752177  88408.01557047 135773.80610257 ...   8384.31040563
 -11459.61661775  12996.37220157]


In [42]:
# Collect testing data form test.csv
test_data = np.genfromtxt('test.csv', delimiter=',', encoding='big5')
test = test_data[ : , 2 : ]

for i in range(test.shape[0]):
    for j in range(test.shape[1]):
        if(math.isnan(test[i][j])):
            test[i][j] = 0.0

test_data = [ ]
daily_data = np.array([])
i = 0
while(i < len(test)):  
    daily_data = np.append(daily_data, test[i])  
    if(i % 18 == 17):
        test_data.append(daily_data.reshape(18 * 9))
        daily_data = np.array([])
    i += 1

test_data = np.array(test_data)
print('Shape of test_data :', test_data.shape)   # 162 = 18 * 9

# Nolmalization
for i in range(test_data.shape[0]):
    for j in range(test_data.shape[1]):
        if not std[j] == 0 :
            test_data[i][j] = (test_data[i][j]- mean[j]) / std[j]

Shape of test_data : (240, 162)


In [43]:
print('Model : y = x * w + b')
b_final = b  # the best-performed b value
w_final = w  # the best-performed w value

# predict the 10-th-hour PM2.5 of test data
test_result = np.array([])
testing_data_prediction = test_data.dot(w_final) + b_final

Model : y = x * w + b


In [51]:
# write the prediction into file
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', 'value'])
    for i in range(len(testing_data_prediction)):
        if (testing_data_prediction[i][0] < 0): testing_data_prediction[i][0] = 0.0
        writer.writerow(['id_' + str(i), testing_data_prediction[i][0]])

In [52]:
testing_data_prediction

array([[  6.57737485],
       [ 18.20787179],
       [ 23.96321068],
       [  7.70659122],
       [ 27.01686971],
       [ 22.25450873],
       [ 23.87509399],
       [ 30.47568852],
       [ 16.77414725],
       [ 59.66249746],
       [ 11.72001981],
       [  9.29460108],
       [ 63.24086518],
       [ 53.24688286],
       [ 22.18996753],
       [ 12.20162222],
       [ 32.40360137],
       [ 66.76522987],
       [  0.        ],
       [ 17.13034548],
       [ 41.78383945],
       [ 72.10541081],
       [  9.30008827],
       [ 17.84397361],
       [ 14.74961255],
       [ 37.79002087],
       [ 14.54740425],
       [ 69.56203906],
       [  7.22113322],
       [ 55.4143094 ],
       [ 24.50568608],
       [  8.49649567],
       [  2.51207549],
       [ 18.67349693],
       [ 27.7569955 ],
       [ 37.21079893],
       [ 43.23896874],
       [ 29.82307804],
       [ 42.20857629],
       [ 35.1461228 ],
       [  7.58154266],
       [ 41.21104654],
       [ 30.57316557],
       [ 50

In [53]:
genre = 1
for i in range(len(w)):
    if(i % 9 == 0):
        print('-\ngenre', genre, ':')
        print('mean =', abs(np.mean(w[i : i + 9])))
        genre += 1
    print('i =', i, ', w =', w[i])

-
genre 1 :
mean = 0.010388553701326261
i = 0 , w = [0.37485267]
i = 1 , w = [-0.84826348]
i = 2 , w = [1.12061649]
i = 3 , w = [-1.661841]
i = 4 , w = [-0.0380905]
i = 5 , w = [1.01189293]
i = 6 , w = [-0.37795467]
i = 7 , w = [-1.68425076]
i = 8 , w = [2.00954133]
-
genre 2 :
mean = 0.04853493666904317
i = 9 , w = [-0.3305962]
i = 10 , w = [0.14193039]
i = 11 , w = [0.07791424]
i = 12 , w = [0.09780359]
i = 13 , w = [-0.02886663]
i = 14 , w = [-0.02437729]
i = 15 , w = [-0.17964756]
i = 16 , w = [0.16481724]
i = 17 , w = [0.51783666]
-
genre 3 :
mean = 0.04260787313879366
i = 18 , w = [0.05241106]
i = 19 , w = [-0.02563757]
i = 20 , w = [0.06613098]
i = 21 , w = [-0.15947092]
i = 22 , w = [0.15258468]
i = 23 , w = [-0.02031218]
i = 24 , w = [-0.16353434]
i = 25 , w = [0.07773467]
i = 26 , w = [0.40356448]
-
genre 4 :
mean = 0.02694439525873901
i = 27 , w = [-0.31887246]
i = 28 , w = [0.37602709]
i = 29 , w = [-0.26119155]
i = 30 , w = [0.29656937]
i = 31 , w = [0.31087021]
i = 32 , w